In [3]:
import logging
logging.basicConfig()
import os
import time

from boxsdk import JWTAuth, Client
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from logging import getLogger, StreamHandler, DEBUG
# チャンネル一覧を取得する
# slack_token = os.environ.get('SLACK_TOKEN')
logger = getLogger(__name__)
client = WebClient(token=slack_token)

In [4]:
TERM=0
channel_id = 'C03HFFXSGUA'
latest = int(time.time() - TERM)  # 現在日時 - 2週間 の UNIX時間
cursor = None  # シーク位置。最初は None ページを指定して、次からは next_cursor が指し示す位置。


try:
    response = client.conversations_history(  # conversations_history ＝ チャット一覧を得る
        channel=channel_id,
        latest=latest,
        cursor=cursor  # チャンネルID、latest、シーク位置を指定。
        # latestに指定した時間よりも古いメッセージが得られる。latestはUNIX時間で指定する。
    )
except SlackApiError as e:
    exit

In [ ]:
'filesがあるかどうかで判定する'
response._initial_data['messages'][0]['files']

In [6]:
response['messages']

[{'type': 'message',
  'text': 'text test',
  'files': [{'id': 'F03LBK1AQMS',
    'created': 1655640254,
    'timestamp': 1655640254,
    'name': 'テキスト.txt',
    'title': 'テキスト.txt',
    'mimetype': 'text/plain',
    'filetype': 'text',
    'pretty_type': 'Plain Text',
    'user': 'U03H0TMS9FH',
    'editable': True,
    'size': 9,
    'mode': 'snippet',
    'is_external': False,
    'external_type': '',
    'is_public': True,
    'public_url_shared': False,
    'display_as_bot': False,
    'username': '',
    'url_private': 'https://files.slack.com/files-pri/T03HFFHAWDQ-F03LBK1AQMS/____________.txt',
    'url_private_download': 'https://files.slack.com/files-pri/T03HFFHAWDQ-F03LBK1AQMS/download/____________.txt',
    'permalink': 'https://slackdevtestenv.slack.com/files/U03H0TMS9FH/F03LBK1AQMS/____________.txt',
    'permalink_public': 'https://slack-files.com/T03HFFHAWDQ-F03LBK1AQMS-680f4413a1',
    'edit_link': 'https://slackdevtestenv.slack.com/files/U03H0TMS9FH/F03LBK1AQMS/_______

# 対象はSlackの指定する3チャンネルにアップロードされている画像(および動画)
# チャンネル一覧を取得する
# fileの一覧を取得する
# {name, upload_date, channel_name(or id)}
# Boxにファイルがあるかを確認する
# ファイルをダウンロード
# 


- Box内のチャンネル名フォルダ配下に日付フォルダを作成し、
- 作成したフォルダにアップロードする
- Slackの容量がいっぱいにならない事を目的に、定期的に実行する
- 一定期間経過したファイルを対象にする(初期値は1か月経過)
- Pythonの自動実行環境の構築はクラウド上に行う想定